In [ ]:
from google.colab import files
uploaded = files.upload()

Saving outPP.txt to outPP.txt


In [ ]:
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding, InputLayer
from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping

In [ ]:
BATCH_SIZE = 32
MIN_WORD_FREQUENCY = 10
SEQUENCE_LEN = 5
EPOCH = 4

In [ ]:
# Data loading and cleaning
with open('outRlyrics.txt', 'r') as content_file:
    content = content_file.read()
songlyrics = content.split("\"")

clean_songlyrics = []
for item in songlyrics:
  if len(item)>1:
    clean_songlyrics.append(item)

Llyrics = []
for l in clean_songlyrics:
  sentences = l.split("\n")
  Llyrics.append(sentences)

Llyrics = [[s for s in l if len(s)>1] for l in Llyrics ]

junks = re.compile(".*?\[(.*?)\]")
bracket = []
for i in range(len(Llyrics)):
  for j in range(len(Llyrics[i])):
    result = re.findall(junks, Llyrics[i][j])
    for item in result:
      if item not in bracket:
        bracket.append(item)
      Llyrics[i][j] = Llyrics[i][j].replace("["+item+"]", "")
    
Llyrics[i] = [s for s in Llyrics[i] if len(s)>1]

for i in range(len(Llyrics)):
  for j in range(len(Llyrics[i])):
    Llyrics[i][j] += "\n"

In [ ]:
text_in_words = [w.lower()   for l in Llyrics for s in l for w in s.split(' ') if len(w)>0]

word_freq = {}
for word in text_in_words:
    word_freq[word] = word_freq.get(word, 0) + 1
    
ignored_words = set()
for k, v in word_freq.items():
    if word_freq[k] < MIN_WORD_FREQUENCY:
        ignored_words.add(k)
        
words = set(text_in_words)
words = sorted(set(words) - ignored_words)

word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

In [ ]:
sentences = []
next_words = []
ignored = 0
for i in range(0, len(text_in_words) - SEQUENCE_LEN, 1):
    # Only add sequences where no word is in ignored_words
    if len(set(text_in_words[i: i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
        sentences.append(text_in_words[i: i + SEQUENCE_LEN])
        next_words.append(text_in_words[i + SEQUENCE_LEN])
    else:
        ignored = ignored+1

In [ ]:
def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    tmp_sentences = []
    tmp_next_word = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))
    return x_train, y_train, x_test, y_test

sentences, next_words, sentences_test, next_words_test = shuffle_and_split_training_set(sentences, next_words)

Size of training set = 662068
Size of test set = 13512


In [ ]:
Lmodel = Sequential()
Lmodel.add(Embedding(input_dim=len(words), output_dim=1024))
Lmodel.add(Bidirectional(LSTM(128)))
Lmodel.add(Dropout(0.2))
Lmodel.add(Dense(len(words)))
Lmodel.add(Activation('softmax'))

In [ ]:
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN), dtype=np.int32)
        y = np.zeros((batch_size), dtype=np.int32)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t] = word_indices[w]
            y[i] = word_indices[next_word_list[index % len(sentence_list)]]
            index = index + 1
        yield x, y

In [ ]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)
    seed_index = np.random.randint(len(sentences+sentences_test))
    seed = (sentences+sentences_test)[seed_index]
    
    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed
        examples_file.write('----- Diversity:' + str(diversity) + '\n')
        examples_file.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
        examples_file.write(' '.join(sentence))
        for i in range(50):
            x_pred = np.zeros((1, SEQUENCE_LEN, len(words)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word_indices[word]] = 1.

            preds = Lmodel.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence = sentence[1:]
            sentence.append(next_word)

            examples_file.write(" "+next_word)
        examples_file.write('\n')
    examples_file.write('='*80 + '\n')
    examples_file.flush()

In [ ]:
#training model

Lmodel.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

file_path = "./checkpoints/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-" \
                "loss{loss:.4f}-acc{acc:.4f}-val_loss{val_loss:.4f}-val_acc{val_acc:.4f}" % \
                (len(words), SEQUENCE_LEN, MIN_WORD_FREQUENCY)

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', save_best_only=True)
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
callbacks_list = [checkpoint, print_callback, early_stopping]

examples_file = open("examples.txt", "w")

Lmodel.fit_generator(generator(sentences, next_words, BATCH_SIZE),
                        steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
                        epochs=EPOCH, #100
                        validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),
                        validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/4
20690/20690 [==============================] - 3918s 189ms/step - loss: 6.1189 - accuracy: 0.0991 - val_loss: 5.1916 - val_accuracy: 0.1719
Epoch 2/4
20690/20690 [==============================] - 3891s 188ms/step - loss: 5.0916 - accuracy: 0.1848 - val_loss: 4.9782 - val_accuracy: 0.2057
Epoch 3/4
20690/20690 [==============================] - 3885s 188ms/step - loss: 4.7251 - accuracy: 0.2231 - val_loss: 4.9178 - val_accuracy: 0.2190
Epoch 4/4
20690/20690 [==============================] - 3806s 184ms/step - loss: 4.4948 - accuracy: 0.2480 - val_loss: 4.8901 - val_accuracy: 0.2295


In [ ]:
def sample(preds, temperature=1.0):
    # Sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def LSTMgenerate(seeding, prefix):
  seed = seeding
  sentence = prefix
  for i in range(20):
    x_pred = np.zeros((len(seeding), len(words)))
    for t, word in enumerate(seed):
      x_pred[t, word_indices[word]] = 1.
    preds = Lmodel.predict(x_pred, verbose=0)[0]
    next_index = sample(preds,10)
    next_word = indices_word[next_index]
    sentence.append(next_word)
    seed = seed[1:]
    seed.append(next_word)
    if "\n" in next_word:
      break
  return sentence

In [ ]:
seed =  ['you','like','i']
prefix = ['you','like','i']
LSoutput = LSTMgenerate(seed, prefix)
LS_sentence = ' '.join(LSoutput)

print(LS_sentence)

you like i play big coupe, scarface right?

